In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import re
import pickle

In [2]:
df=pd.read_csv('data/final_df.csv')
df

,Unnamed: 0,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,product_information,...,lower_price,upper_price,avg_price,price_tag,seller_info_list,product_name_tokens,seller_list_tokens,product_info_token,combined_tokens,main_categories
0,0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,5,15,1,4.9,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Item Weight640 g Product Dim...,...,3.42,3.42,3.42,0-25,"Amazon.co.uk,£3.42,**stop-&-shop-uk**,£0.19,Wo...","Hornby,2014,Catalogue","Amazon.co.uk,**stop-&-shop-uk**,World Wide Sho...","Technical,Details,Item,Weight640,Product,Dimen...","Hornby,2014,CatalogueHobbies , Model Trains & ...",Hobbies
1,1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,0,2,1,4.5,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Manufacturer recommended age...,...,16.99,16.99,16.99,0-25,"UHD WHOLESALE,£16.99","FunkyBuys®,Large,Christmas,Holiday,Express,Fes...",UHD WHOLESALE,"Technical,Details,Manufacturer,recommended,age...","FunkyBuys®,Large,Christmas,Holiday,Express,Fes...",Hobbies
2,2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,2,17,2,3.9,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Manufacturer recommended age...,...,9.99,9.99,9.99,0-25,"DEAL-BOX,£9.99,DO NOT BUY - TESTING ACCOUNT,£2...","CLASSIC,TOY,TRAIN,SET,TRACK,CARRIAGES,LIGHT,EN...","DEAL-BOX,DO NOT BUY - TESTING ACCOUNT","Technical,Details,Manufacturer,recommended,age...","CLASSIC,TOY,TRAIN,SET,TRACK,CARRIAGES,LIGHT,EN...",Hobbies
3,3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,0,1,2,5.0,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Item Weight259 g Product Dim...,...,39.99,39.99,39.99,25-50,no_seller_info,"HORNBY,Coach,R4410A,BR,Hawksworth,Corridor,3rd",no_seller_info,"Technical,Details,Item,Weight259,Product,Dimen...","HORNBY,Coach,R4410A,BR,Hawksworth,Corridor,3rd...",Hobbies
4,4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,0,3,2,4.7,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Item Weight159 g Product Dim...,...,32.19,32.19,32.19,25-50,no_seller_info,"Hornby,00,Gauge,0-4-0,Gildenlow,Salt,Co,Steam,...",no_seller_info,"Technical,Details,Item,Weight159,Product,Dimen...","Hornby,00,Gauge,0-4-0,Gildenlow,Salt,Co,Steam,...",Hobbies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,44d6967f083825a5de36ad4865a65bcd,Batman 1966 TV Series Action Figures - The Rid...,Mattel,5,3,3,5.0,"Hobbies , Collectible Figures & Memorabilia , ...",Technical Details Item Weight136 g Product Dim...,...,22.95,22.95,22.95,0-25,"Star Action Figures,£22.95,Order66ToyStore,£19...","Batman,1966,TV,Series,Action,Figures,The,Riddler","Star Action Figures,Order66ToyStore,X-Comics G...","Technical,Details,Item,Weight136,Product,Dimen...","Batman,1966,TV,Series,Action,Figures,The,Riddl...",Hobbies
9996,9996,08f0747b6fc6687215ffb994c3a6fb32,"Star Wars Costume, Kids Stormtrooper Costume S...",Star Wars,0,1,3,4.0,"Characters & Brands , Star Wars , Toys",Technical Details Additional Information AS...,...,39.99,39.99,39.99,25-50,no_seller_info,"Star,Wars,Costume,Kids,Stormtrooper,Costume,St...",no_seller_info,"Technical,Details,Additional,Information,ASINB...","Star,Wars,Costume,Kids,Stormtrooper,Costume,St...",Characters
9997,9997,bf6cc073f8f24e6e338190fa16f6ee9d,Defiance Lawkeeper Metal Badge Prop Replica,Olde Scotland Yard Ltd.,3,1,3,5.0,"Novelty & Special Use , Novelty , Accessories ...",Technical Details Item Weight159 g Product Dim...,...,43.99,43.99,43.99,25-50,"YUK,£43.99,DealsCentre,£116.50,DALY store,£124.95","Defiance,Lawkeeper,Metal,Badge,Prop,Replica","YUK,DealsCentre,DALY store","Technical,Details,Item,Weight159,Pro

In [3]:
df.columns

Index(['Unnamed: 0', 'uniq_id', 'product_name', 'manufacturer',
       'number_available_in_stock', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'amazon_category_and_sub_category', 'product_information',
       'customer_questions_and_answers', 'customer_reviews', 'lower_price',
       'upper_price', 'avg_price', 'price_tag', 'seller_info_list',
       'product_name_tokens', 'seller_list_tokens', 'product_info_token',
       'combined_tokens', 'main_categories'],
      dtype='object')

In [4]:
sorted(df['price_tag'].value_counts().index.tolist())

['0-25', '100-200', '25-50', '50-100', '>200', 'No Price']

In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['combined_tokens'])
cos = cosine_similarity(X)

In [6]:
model = pickle.load(open('cos_similarity.pkl','rb'))

In [7]:
model.shape

(10000, 10000)

In [8]:
type(model)

numpy.ndarray

In [9]:
makeup_id = df[df['product_name']=='Hornby 2014 Catalogue'].index.values[0]
makeup_id

0

In [10]:
scores = list(enumerate(model[makeup_id]))
scores

[(0, 1.0000000000000002),
 (1, 0.22562957225936708),
 (2, 0.2842535156701291),
 (3, 0.29892623306062116),
 (4, 0.3361729550684171),
 (5, 0.1481668435428137),
 (6, 0.3555520745494467),
 (7, 0.22795345427097038),
 (8, 0.3753085945200956),
 (9, 0.33329742478841856),
 (10, 0.409571885792173),
 (11, 0.2972539737610202),
 (12, 0.3242413409738872),
 (13, 0.3287958444524553),
 (14, 0.34715918995122835),
 (15, 0.12232540195866244),
 (16, 0.4338647753355104),
 (17, 0.31785045880295276),
 (18, 0.2269112217483135),
 (19, 0.3100345211699448),
 (20, 0.3708876488560928),
 (21, 0.20677579300984267),
 (22, 0.3156146493224431),
 (23, 0.2637093163162526),
 (24, 0.35638916469751647),
 (25, 0.3328707711827446),
 (26, 0.2724429805430632),
 (27, 0.36816802853602715),
 (28, 0.21431914030221158),
 (29, 0.15761598838407204),
 (30, 0.2931996198642419),
 (31, 0.3483518173046799),
 (32, 0.27179722178067706),
 (33, 0.13387608615483682),
 (34, 0.22670652515810946),
 (35, 0.22992159710678267),
 (36, 0.422023120672240

In [11]:
sorted_scores = scores[1:]
sorted_scores


[(1, 0.22562957225936708),
 (2, 0.2842535156701291),
 (3, 0.29892623306062116),
 (4, 0.3361729550684171),
 (5, 0.1481668435428137),
 (6, 0.3555520745494467),
 (7, 0.22795345427097038),
 (8, 0.3753085945200956),
 (9, 0.33329742478841856),
 (10, 0.409571885792173),
 (11, 0.2972539737610202),
 (12, 0.3242413409738872),
 (13, 0.3287958444524553),
 (14, 0.34715918995122835),
 (15, 0.12232540195866244),
 (16, 0.4338647753355104),
 (17, 0.31785045880295276),
 (18, 0.2269112217483135),
 (19, 0.3100345211699448),
 (20, 0.3708876488560928),
 (21, 0.20677579300984267),
 (22, 0.3156146493224431),
 (23, 0.2637093163162526),
 (24, 0.35638916469751647),
 (25, 0.3328707711827446),
 (26, 0.2724429805430632),
 (27, 0.36816802853602715),
 (28, 0.21431914030221158),
 (29, 0.15761598838407204),
 (30, 0.2931996198642419),
 (31, 0.3483518173046799),
 (32, 0.27179722178067706),
 (33, 0.13387608615483682),
 (34, 0.22670652515810946),
 (35, 0.22992159710678267),
 (36, 0.42202312067224024),
 (37, 0.3327974452777

In [12]:
sorted_scores = sorted_scores[:10]
sorted_scores

[(1, 0.22562957225936708),
 (2, 0.2842535156701291),
 (3, 0.29892623306062116),
 (4, 0.3361729550684171),
 (5, 0.1481668435428137),
 (6, 0.3555520745494467),
 (7, 0.22795345427097038),
 (8, 0.3753085945200956),
 (9, 0.33329742478841856),
 (10, 0.409571885792173)]

In [13]:
a = [(1,321),(1,432),(2,21),(43,33),(3,412)]
b = [2,3]

x = [i[0] for i in a]
y = [a[x.index(j)] for j in b if j in x]
# for j in b:
#     if j in x:
#         y.append(x.index(j))
y
for idx in y:
    a.remove(idx)
print(a)

[(1, 321), (1, 432), (43, 33)]


In [14]:
a = [1,2,3,4]
b = [1,4]
for i in b:
    a.remove(i)
print(a)

[2, 3]


In [15]:
# filter brand name
def filter(brand_name,price_range):
    filter_index = df.loc[(df['manufacturer']!=brand_name)&(df['price_tag']!=price_range)].index
    return filter_index.values
    



In [16]:
makeup_id = df[df['product_name']=='Hornby 2014 Catalogue'].index.values[0]
scores = list(enumerate(model[makeup_id]))
scores

[(0, 1.0000000000000002),
 (1, 0.22562957225936708),
 (2, 0.2842535156701291),
 (3, 0.29892623306062116),
 (4, 0.3361729550684171),
 (5, 0.1481668435428137),
 (6, 0.3555520745494467),
 (7, 0.22795345427097038),
 (8, 0.3753085945200956),
 (9, 0.33329742478841856),
 (10, 0.409571885792173),
 (11, 0.2972539737610202),
 (12, 0.3242413409738872),
 (13, 0.3287958444524553),
 (14, 0.34715918995122835),
 (15, 0.12232540195866244),
 (16, 0.4338647753355104),
 (17, 0.31785045880295276),
 (18, 0.2269112217483135),
 (19, 0.3100345211699448),
 (20, 0.3708876488560928),
 (21, 0.20677579300984267),
 (22, 0.3156146493224431),
 (23, 0.2637093163162526),
 (24, 0.35638916469751647),
 (25, 0.3328707711827446),
 (26, 0.2724429805430632),
 (27, 0.36816802853602715),
 (28, 0.21431914030221158),
 (29, 0.15761598838407204),
 (30, 0.2931996198642419),
 (31, 0.3483518173046799),
 (32, 0.27179722178067706),
 (33, 0.13387608615483682),
 (34, 0.22670652515810946),
 (35, 0.22992159710678267),
 (36, 0.422023120672240

In [17]:
len(filter('FunkyBuys','below_or_equal_twentyfive'))

9998

In [18]:
x = [i[0] for i in scores]
y = [scores[x.index(j)] for j in filter('FunkyBuys','below_or_equal_twentyfive') if j in x]
y

[(0, 1.0000000000000002),
 (2, 0.2842535156701291),
 (3, 0.29892623306062116),
 (4, 0.3361729550684171),
 (5, 0.1481668435428137),
 (6, 0.3555520745494467),
 (7, 0.22795345427097038),
 (8, 0.3753085945200956),
 (9, 0.33329742478841856),
 (10, 0.409571885792173),
 (11, 0.2972539737610202),
 (12, 0.3242413409738872),
 (13, 0.3287958444524553),
 (14, 0.34715918995122835),
 (15, 0.12232540195866244),
 (16, 0.4338647753355104),
 (17, 0.31785045880295276),
 (18, 0.2269112217483135),
 (19, 0.3100345211699448),
 (20, 0.3708876488560928),
 (21, 0.20677579300984267),
 (22, 0.3156146493224431),
 (23, 0.2637093163162526),
 (24, 0.35638916469751647),
 (25, 0.3328707711827446),
 (26, 0.2724429805430632),
 (27, 0.36816802853602715),
 (28, 0.21431914030221158),
 (29, 0.15761598838407204),
 (30, 0.2931996198642419),
 (31, 0.3483518173046799),
 (32, 0.27179722178067706),
 (33, 0.13387608615483682),
 (34, 0.22670652515810946),
 (35, 0.22992159710678267),
 (36, 0.42202312067224024),
 (37, 0.33279744527776

In [19]:
sorted_scores = sorted(y, key = lambda x: x[1], reverse=True)
sorted_scores[:10]

[(0, 1.0000000000000002),
 (5734, 0.46146628863074857),
 (5764, 0.4598440686798619),
 (46, 0.44167139380016035),
 (179, 0.4415990230151961),
 (123, 0.4385326277684315),
 (16, 0.4338647753355104),
 (125, 0.4285142691494722),
 (36, 0.42202312067224024),
 (5782, 0.4204557067049584)]

In [20]:
# ## not included
# def exclude_selection(brand_name,price_range):
#     exclude_index = df.loc[(df['manufacturer']!=brand_name)&(df['price_tag']==price_range)].index.values
#     print(exclude_index[0:10])
#     return exclude_index
    
# ## included
# def include_selection(brand_name,price_range):
#     include_index = df.loc[(df['manufacturer']==brand_name)&(df['price_tag']==price_range)].index.values
#     print(include_index[0:10])
#     return include_index
    
# ## different execution considering the checkbox checked or not
# def decision(brand_name,price_range):
#     included = 1 # 1 = yes, 0 = no
#     if included:
#         include_selection(brand_name,price_range)
#     else:
#         exclude_selection(brand_name,price_range)

# def filter_product(scores, brand_name,price_range):
#     x = [i[0] for i in scores]
#     y = [scores[x.index(j)] for j in decision(brand_name,price_range) if j in x]
#     print(y[:10])
#     return y




In [21]:
## different execution considering the checkbox checked or not

def decision(included,brand_name,price_range):
    included = included # 1 = yes, 0 = no
    if included:
        include_index = df.loc[(df['manufacturer']==brand_name)&(df['price_tag']==price_range)].index.values
    else:
        include_index = df.loc[(df['manufacturer']!=brand_name)&(df['price_tag']==price_range)].index.values
    return include_index

def filter_product(included,scores, brand_name,price_range):
    x = [i[0] for i in scores]
    y = [scores[x.index(j)] for j in decision(included,brand_name,price_range) if j in x]
    print(y[:10])
    return y

In [22]:
def recommend(name,included,price_range): 
    recommend_list = []
    try:
        makeup_id = df[df['product_name'].str.contains(name, case=False)].index.values[0]
    except:
        try: 
            makeup_id = df[df['manufacture'].str.contains(name, case=False)].index.values[0]
        except:
            try:
                makeup_id = df[df['categories'].str.contains(name, case=False)].index.values[0]
            except:
                makeup_id = max(enumerate([ls.count(name) for ls in df['combined_tokens']]), key=lambda x: x[1])[0]
    
    brand_name = df[df['product_name']=='Hornby 2014 Catalogue']['manufacturer'][0]

    scores = list(enumerate(model[makeup_id]))
    print(scores[:10])
    y = filter_product(included,scores,brand_name,price_range)
    sorted_scores = sorted(y, key = lambda x: x[1], reverse=True)
    sorted_scores = sorted_scores[:10]
    print(sorted_scores)
    

    j = 0
    print('The 10 most recommened products: ')
    for item in sorted_scores:
        product_name = df.iloc[item[0],:]['product_name']
        recommend_list.append(product_name)
        j = j+1
        if j > 9:
            break
    return recommend_list

In [23]:
selected = df[df['product_name'].str.contains('car', case=False)]['product_name']

In [24]:
recommend('Hornby 2014 Catalogue',0,'below_or_equal_twentyfive')

[(0, 1.0000000000000002), (1, 0.22562957225936708), (2, 0.2842535156701291), (3, 0.29892623306062116), (4, 0.3361729550684171), (5, 0.1481668435428137), (6, 0.3555520745494467), (7, 0.22795345427097038), (8, 0.3753085945200956), (9, 0.33329742478841856)]
[]
[]
The 10 most recommened products: 


[]

In [25]:
recommend('car',1,'below_or_equal_twentyfive')

[(0, 0.2842535156701291), (1, 0.317503238533758), (2, 1.0000000000000002), (3, 0.31995489127618054), (4, 0.3313464782183147), (5, 0.2001771448324433), (6, 0.3049663094762696), (7, 0.17611551292270192), (8, 0.3094224192754407), (9, 0.2442860414423428)]
[]
[]
The 10 most recommened products: 


[]

In [26]:
df[(df['product_name']!="Hornby 2014 Catalogue")&(df['manufacturer']!='Hornby')&(df['price_tag']=='below_or_equal_twentyfive')].index.values

array([], dtype=int64)

In [27]:
makeup_id = df[df['product_name']=='Hornby 2014 Catalogue'].index.values[0]
scores = list(enumerate(model[makeup_id]))
scores

[(0, 1.0000000000000002),
 (1, 0.22562957225936708),
 (2, 0.2842535156701291),
 (3, 0.29892623306062116),
 (4, 0.3361729550684171),
 (5, 0.1481668435428137),
 (6, 0.3555520745494467),
 (7, 0.22795345427097038),
 (8, 0.3753085945200956),
 (9, 0.33329742478841856),
 (10, 0.409571885792173),
 (11, 0.2972539737610202),
 (12, 0.3242413409738872),
 (13, 0.3287958444524553),
 (14, 0.34715918995122835),
 (15, 0.12232540195866244),
 (16, 0.4338647753355104),
 (17, 0.31785045880295276),
 (18, 0.2269112217483135),
 (19, 0.3100345211699448),
 (20, 0.3708876488560928),
 (21, 0.20677579300984267),
 (22, 0.3156146493224431),
 (23, 0.2637093163162526),
 (24, 0.35638916469751647),
 (25, 0.3328707711827446),
 (26, 0.2724429805430632),
 (27, 0.36816802853602715),
 (28, 0.21431914030221158),
 (29, 0.15761598838407204),
 (30, 0.2931996198642419),
 (31, 0.3483518173046799),
 (32, 0.27179722178067706),
 (33, 0.13387608615483682),
 (34, 0.22670652515810946),
 (35, 0.22992159710678267),
 (36, 0.422023120672240

In [28]:
x = [i[0] for i in scores]
print('x: {}'.format(x[0:10]))
# 10000 data


x: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [29]:
selected_index = df.loc[(df['manufacturer']!='Hornby')&(df['price_tag']=='below_or_equal_twentyfive')].index.values
print(selected_index[0:10])

[]


In [30]:
y = [scores[x.index(j)] for j in selected_index if j in x]
print('y: {}'.format(y[:10]))
print('len: {}'.format(len(y)))

y: []
len: 0


In [31]:
sorted_scores = sorted(y, key = lambda x: x[1], reverse=True)
sorted_scores[:10]

[]

In [32]:
j = 0
print('The 10 most recommened products: ')
for item in sorted_scores:
    product_name = df.iloc[item[0],:]['product_name']
    print(j+1, product_name)
    j = j+1
    if j > 9:
        break

The 10 most recommened products: 


In [33]:
df[df['product_name']=='Hornby 2014 Catalogue']['manufacturer'][0]

'Hornby'

In [34]:
price_tags = df['price_tag'].unique().tolist()

In [35]:
def price_tag_converter():
    price_list=[]
    price_dict= {
        'below_or_equal_twentyfive': '0-25',
        'twentyfive_to_fifty': '25-50',
        'fifty_to_hundred': '50-100',
        'one_to_two_hundred': '100-200',
        'two_hundred_above': '>200',
        'no_price': 'No Price'
        }
    price_list = [price_dict[price_tag] for price_tag in price_tags if price_tag in price_dict]
    return sorted(price_list)
price_tag_converter()

[]

In [36]:
price_list = price_tag_converter()
print(price_list)

[]


In [37]:
price_dict= {
        'below_or_equal_twentyfive': '0-25',
        'twentyfive_to_fifty': '25-50',
        'fifty_to_hundred': '50-100',
        'one_to_two_hundred': '100-200',
        'two_hundred_above': '>200',
        'no_price': 'No Price'
        }

In [38]:
price = '0-25'
price = [i for i ,r in price_dict.items() if price == r][0]

In [39]:
price

'below_or_equal_twentyfive'

In [40]:
a = df['amazon_category_and_sub_category'][0]
type(a)

str

In [41]:
import re
pattern = re.compile(r'\s.+')
main_categories = df['amazon_category_and_sub_category'].apply(lambda x: re.sub(pattern,'',x))
df['main_categories'] = main_categories
categories_list = list(main_categories.value_counts().index)

In [42]:
## Only show top 20 brand on particular category
def filtered_brand(selected_cat):
    filtered_brand = list(df[df['main_categories']=='Cooking']['manufacturer'].value_counts().sort_values(ascending=False).head(20))
    return filtered_brand

In [43]:
list(df[df['main_categories']=='Cooking']['manufacturer'].value_counts().sort_values(ascending=False).head(20))

[1, 1, 1, 1]

In [44]:
df['combined_tokens']

0       Hornby,2014,CatalogueHobbies , Model Trains & ...
1       FunkyBuys®,Large,Christmas,Holiday,Express,Fes...
2       CLASSIC,TOY,TRAIN,SET,TRACK,CARRIAGES,LIGHT,EN...
3       HORNBY,Coach,R4410A,BR,Hawksworth,Corridor,3rd...
4       Hornby,00,Gauge,0-4-0,Gildenlow,Salt,Co,Steam,...
                              ...                        
9995    Batman,1966,TV,Series,Action,Figures,The,Riddl...
9996    Star,Wars,Costume,Kids,Stormtrooper,Costume,St...
9997    Defiance,Lawkeeper,Metal,Badge,Prop,ReplicaNov...
9998    Justice,League,America,Series,3,Green,Lantern,...
9999    Star,Wars,1/72,Y-Wing,StarfighterCharacters & ...
Name: combined_tokens, Length: 10000, dtype: object

In [45]:
df[df['manufacturer']=='Men']

,Unnamed: 0,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,product_information,...,lower_price,upper_price,avg_price,price_tag,seller_info_list,product_name_tokens,seller_list_tokens,product_info_token,combined_tokens,main_categories


In [97]:
def keywords(name): 
    try:
        keywords_product = df[df['product_name'].str.contains(name, case=False)]['product_name'].head(20)
    except:
        try: 
            keywords_product  = df[df['manufacture'].str.contains(name, case=False)]['product_name'].head(20)
        except:
            try:
                keywords_product  = df[df['categories'].str.contains(name, case=False)]['product_name'].head(20)
            except:
                return 'No suitable products'
    return keywords_product.values.tolist()
    # brand_name = df[df['product_name']=='Hornby 2014 Catalogue']['manufacturer'][0]

    # scores = list(enumerate(model[makeup_id]))
    # print(scores[:10])
    # y = filter_product(included,scores,brand_name,price_range)
    # sorted_scores = sorted(y, key = lambda x: x[1], reverse=True)
    # sorted_scores = sorted_scores[:10]
    # print(sorted_scores)
    

    # j = 0
    # print('The 10 most recommened products: ')
    # for item in sorted_scores:
    #     product_name = df.iloc[item[0],:]['product_name']
    #     recommend_list.append(product_name)
    #     j = j+1
    #     if j > 9:
    #         break
    # return recommend_list

In [104]:
df[(df['product_name'].str.contains('car', case=False))&(df['price_tag']=='0-25')]

,Unnamed: 0,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,product_information,...,lower_price,upper_price,avg_price,price_tag,seller_info_list,product_name_tokens,seller_list_tokens,product_info_token,combined_tokens,main_categories
2,2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,2,17,2,3.9,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Manufacturer recommended age...,...,9.99,9.99,9.99,0-25,"DEAL-BOX,£9.99,DO NOT BUY - TESTING ACCOUNT,£2...","CLASSIC,TOY,TRAIN,SET,TRACK,CARRIAGES,LIGHT,EN...","DEAL-BOX,DO NOT BUY - TESTING ACCOUNT","Technical,Details,Manufacturer,recommended,age...","CLASSIC,TOY,TRAIN,SET,TRACK,CARRIAGES,LIGHT,EN...",Hobbies
86,86,4b15acfc7dc8ff73020661531b9c3a3d,Thomas and Friends Track Master Dockside Deliv...,Mattel,6,1,1,5.0,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Item Weight141 g Product Dim...,...,13.87,13.87,13.87,0-25,"JemToys,£13.87,Toys Games and Collectables,£15...","Thomas,Friends,Track,Master,Dockside,Delivery,...","JemToys,Toys Games and Collectables,OrchardCre...","Technical,Details,Item,Weight141,Product,Dimen...","Thomas,Friends,Track,Master,Dockside,Delivery,...",Hobbies
107,107,bb2466d62f3fdc9510272ee5233694d3,Walthers Trainline 931-1676 TL 50' PD Boxcar ATSF,Walthers Trainline,2,1,1,5.0,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Item Weight45 g Product Dime...,...,15.00,15.00,15.00,0-25,"Gaugemaster,£15.00,haupteingang1,£11.46","Walthers,Trainline,931-1676,TL,50,PD,Boxcar,ATSF","Gaugemaster,haupteingang1","Technical,Details,Item,Weight45,Product,Dimens...","Walthers,Trainline,931-1676,TL,50,PD,Boxcar,AT...",Hobbies
145,145,8df980439240865028b8808ef7b14cb8,Preiser 30468 Horse Drawn Open Farm Cart,Preiser,4,1,2,5.0,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Item Weight100 g Product Dim...,...,20.21,20.21,20.21,0-25,"Au Pullman,£20.21,tablet_books,£40.99,francejo...","Preiser,30468,Horse,Drawn,Open,Farm,Cart","Au Pullman,tablet_books,francejouet,Sam Store US","Technical,Details,Item,Weight100,Product,Dimen...","Preiser,30468,Horse,Drawn,Open,Farm,CartHobbie...",Hobbies
149,149,3771983ef8f14ef1b7738c31a2c05856,Plan Toys Cargo Train,Plan Toys,8,1,2,5.0,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Item Weight45 g Product Dime...,...,15.83,15.83,15.83,0-25,"Morleys,£9.99,Gameseek,£15.51,Amazon.co.uk,£15...","Plan,Toys,Cargo,Train","Morleys,Gameseek,Amazon.co.uk,Otherland UK,Ser...","Technical,Details,Item,Weight45,Product,Dimens...","Plan,Toys,Cargo,TrainHobbies , Model Trains & ...",Hobbies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9787,9787,2c088f60dcb8876aa5a7c9e058cd0abb,Ultra Pro SLEEVES Pro-Fit Clear C100 Card Game...,Ultra Pro,9,1,0,5.0,"Party Supplies , Decorations , Balloons",Technical Details Item Weight45 g Product Dime...,...,4.61,4.61,4.61,0-25,"cardandclix,£4.59,Amazon.co.uk,£4.61,GamesQues...","Ultra,Pro,SLEEVES,Pro-Fit,Clear,C100,Card,Game...","cardandclix,Amazon.co.uk,GamesQuest,Beat The B...","Technical,Details,Item,Weight45,Product,Dimens...","Ultra,Pro,SLEEVES,Pro-Fit,Clear,C100,Card,Game...",Party
9805,9805,292b7e96f6db61394191bc346f3915cc,Ultra Pro SLEEVES Pro-Matte d12 Card Game (Lim...,Ultra Pro,12,16,1,4.4,"Hobbies , Trading Cards & Accessories , Storag...",Technical Details Item Weight41 g Product Dime...,...,2.49,2.49,2.49,0-25,"Amazon.co.uk,£2.49,Legendgames,£2.49,Fun Colle...","Ultra,Pro,SLEEVES,Pro-Matte,d12,Card,Game,Lime...","Amazon.co.uk,Legendgames,Fun Collectables,Funa...","Technical,Details,Item,Weight41,Product,Dimens...","Ultra,Pro,SLEEVES,Pro-Matte,d12,Card,Game,Lime...",Hobbies
9874,9874,c03b39c07ebb45ded377c57e74fb04a3,Ultra Pro SLEEVES MTG Mana 4 Planeswalkers Gar...,Ultra Pro,14,1,1,5.0,"Hobbies , Trading Cards & Accessorie

In [57]:
a = keywords('car')
a

['CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT ENGINE BOXED BOYS KIDS BATTERY',
 'Glacier Express of N gauge 10-1219 Alps [UNESCO paint color] 7 Car Set',
 'Alps Glacier Express (Add-On 4-Car Set) (Model Train)',
 'Ohigawa Railway SL [Kawane-ji] (4-Car Set) (Model Train)',
 'Kato 10-847 E259 Narita Express 3 Car Powered Set',
 'Subway train 500 form three-car basic set red gauge N 10-1134 Marunouchi Line',
 'Thomas and Friends Track Master Dockside Delivery Crane Cargo and Cars Set',
 'Graham Farish 374-807 BR Mk1 RFO Restaurant Car Crimson & Cream',
 'Hornby R3162A BR 2-BIL 2 Car EMU Pack',
 'Carousel Wooden Train Set (100 Piece)',
 "Walthers Trainline 931-1676 TL 50' PD Boxcar ATSF",
 'Series 225-5000 [Kansai Airport, Kishu-ji Rapid Service] (4-Car Set) (Model Train)',
 'N gauge 10-1221 E7 system Hokuriku Shinkansen basic set (3 cars)',
 'BR Mk1 SLF Sleeper Car Maroon',
 'Hornby R2987 Brighton Belle 1934 00 Gauge DCC Ready 2 Car Pack',
 'Kato 10-809 Freight Train 6 Car Set',
 'Preiser

In [71]:
df[df['product_name']=='CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT ENGINE BOXED BOYS KIDS BATTERY']

,Unnamed: 0,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,product_information,...,lower_price,upper_price,avg_price,price_tag,seller_info_list,product_name_tokens,seller_list_tokens,product_info_token,combined_tokens,main_categories
2,2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,2,17,2,3.9,"Hobbies , Model Trains & Railway Sets , Rail V...",Technical Details Manufacturer recommended age...,...,9.99,9.99,9.99,0-25,"DEAL-BOX,£9.99,DO NOT BUY - TESTING ACCOUNT,£2...","CLASSIC,TOY,TRAIN,SET,TRACK,CARRIAGES,LIGHT,EN...","DEAL-BOX,DO NOT BUY - TESTING ACCOUNT","Technical,Details,Manufacturer,recommended,age...","CLASSIC,TOY,TRAIN,SET,TRACK,CARRIAGES,LIGHT,EN...",Hobbies


In [92]:
a =sorted(df['price_tag'].value_counts().index.tolist())
a.pop(-1)
print(a)

['0-25', '100-200', '25-50', '50-100', '>200']


In [81]:
manufacturer_list=[df[df['product_name']==i]['manufacturer'].values[0] for i in a]
price_list=[df[df['product_name']==i]['price_tag'].values[0] for i in a]
price_list

IndexError: index 0 is out of bounds for axis 0 with size 0

In [48]:
def advance_search_recommend(name): 
    recommend_list = []
    makeup_id = df[df['product_name']==name].index.values[0]
    scores = list(enumerate(model[makeup_id]))
    print(scores[:10])
    sorted_scores = sorted(scores, key = lambda x: x[1], reverse=True)
    sorted_scores = sorted_scores[:10]
    print(sorted_scores)
    
    j = 0
    print('The 10 most recommened products: ')
    for item in sorted_scores:
        product_name = df.iloc[item[0],:]['product_name']
        recommend_list.append(product_name)
        j = j+1
        if j > 9:
            break
    return recommend_list

In [49]:
advance_search_recommend('Glacier Express of N gauge 10-1219 Alps [UNESCO paint color] 7 Car Set')

[(0, 0.12232540195866244), (1, 0.12244971116012508), (2, 0.13988770060326888), (3, 0.14656826702560538), (4, 0.15755235168918613), (5, 0.07120798624959299), (6, 0.15051461694337787), (7, 0.7389744979707595), (8, 0.14463726141307295), (9, 0.10501199350516348)]
[(15, 1.0000000000000002), (143, 0.7773784270258777), (5728, 0.7600438581145035), (5785, 0.7557818724435015), (174, 0.7534337428143117), (52, 0.7531174045737192), (110, 0.7513784390229155), (5725, 0.7511525822078796), (5783, 0.7477535109941834), (7792, 0.7476944599786005)]
The 10 most recommened products: 


['Glacier Express of N gauge 10-1219 Alps [UNESCO paint color] 7 Car Set',
 'Kato 10-809 Freight Train 6 Car Set',
 'Trix 22783',
 'Disney Planes "2" Muir - Deluxe Die Cast (Metal) Train',
 'Lelin Wooden Train Track Railway Station Playset Childrens Kids Pretend Play',
 'Hornby Digital 15 V 4 Amp Transformer',
 'Lightweight HyDroc',
 'Knightwing Model Railway 0-4-0 Diesel Shunter Locomotive - OO Gauge Plastic Kit',
 'Hornby The Western Spirit OO Gauge Electric Train Set',
 'Bigjigs Rail Heritage Collection Flying Scotsman']

In [50]:
df[df['product_name']=='MINI COOPER CAR TOY KIDS CLASSIC UNION JACK UK DIE CAST SCALE CAR MODEL GIFT NEW']

,Unnamed: 0,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,product_information,...,lower_price,upper_price,avg_price,price_tag,seller_info_list,product_name_tokens,seller_list_tokens,product_info_token,combined_tokens,main_categories
7615,7615,f6d56c2504544875a21a7509b3905cf0,MINI COOPER CAR TOY KIDS CLASSIC UNION JACK UK...,BARGAINS-GALORE,2,13,1,4.2,"Die-Cast & Toy Vehicles , Toy Vehicles & Acces...",Technical Details Manufacturer recommended age...,...,4.49,4.49,4.49,0-25,"GBP International,£4.49,BARGAINS-GALORE,£4.49","MINI,COOPER,CAR,TOY,KIDS,CLASSIC,UNION,JACK,UK...","GBP International,BARGAINS-GALORE","Technical,Details,Manufacturer,recommended,age...","MINI,COOPER,CAR,TOY,KIDS,CLASSIC,UNION,JACK,UK...",Die-Cast


In [51]:
a = 'MINI,COOPER,CAR,TOY,KIDS,CLASSIC,UNION,JACK,UK,DIE,CAST,SCALE,CAR,MODEL,GIFT,NEWDie-Cast & Toy Vehicles , Toy Vehicles & Accessories , Scaled Models , Vehiclesbelow_or_equal_twentyfiveGBP International,BARGAINS-GALORETechnical,Details,Manufacturer,recommended,age:3,years,Additional,Information,ASINB00K853OJG,Best,Sellers,Rank,Toys,Games,See,top,Delivery,Destinations,Visit,Delivery,Destinations,Help,page,see,item,delivered,Date,First,Available9,May,Feedback,Would,like,update,product,info,give,feedback,images'

In [52]:
a = ['a','b','c','d']
b = ['b','d']
for i in b:
    print(i in a)

True
True


In [53]:
for keywords in input1.lower().split():
    if keywords in a:
        count = count

NameError: name 'input1' is not defined

In [ ]:
a.lower().split(',')

In [ ]:
df['combined_tokens'][0]

In [ ]:
df[df['combined_tokens'].str.contains('car toy', case=False)]

In [ ]:
df[df['product_name']=='Kato 23-233 Wooden Station & Buildings']
# manufacturer = df[df['product_name']=='Kato 23-233 Wooden Station & Buildings']['manufacturer']
# manufacturer = df[df['product_name']=='Kato 23-233 Wooden Station & Buildings']['manufacturer']